In [ ]:
name_map = {
    "llama.v1.0":"TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "llama.v1.1":"TinyLlama/TinyLlama_v1.1",
    "llama.math":"TinyLlama/TinyLlama_v1.1_math_code",
    "v1.0":"JanDkff/TinyFuncCoder-v1.0",
    "v1.1":"JanDkff/TinyFuncCoder-v1.1"
}


In [ ]:

#evaluate_finetuned = False

reformatted_prompt = False
reformatted_path = "R" if reformatted_prompt else ""


eval_types=["humaneval","mbpp","mbppplus","multiple-sh","multiple-cpp","multiple-cs","multiple-js","multiple-java","multiple-php","multiple-ts"] #,"humanevalsynthesize-python","humanevalsynthesize-cpp","humanevalsynthesize-js","humanevalsynthesize-java","ds1000-all-completion"]

#names = ["v1.0","v1.1"] if evaluate_finetuned else ["llama.v1.0","llama.v1.1","llama.math"]

names = ["llama.v1.0"]

eval_types = ["ds1000-all-completion"]

In [ ]:
# GENERATE DS-1000
from transformers import AutoModelForCausalLM, pipeline, AutoTokenizer
from peft import PeftModelForCausalLM
from datasets import load_dataset
import json

data = load_dataset("xlangai/DS-1000",split="test")

for name in names:
    if name.find("llama") >= 0:
        model = AutoModelForCausalLM.from_pretrained(name_map[name])
        tokenizer = AutoTokenizer.from_pretrained(name_map[name])
    else:
        model = AutoModelForCausalLM.from_pretrained(name_map[f"llama.{name}"])
        peft = PeftModelForCausalLM.from_pretrained(model=model,model_id=name_map[name])
        tokenizer = AutoTokenizer.from_pretrained(name_map[f"llama.{name}"])
        model = peft.merge_and_unload()
    generator = pipeline(task="text-generation",model=model,tokenizer=tokenizer, max_new_tokens=512,device="cuda")
    
    new_data = []
    i = 1
    for problem in data:
        #print(problem)
        print(f'{name}: {i}/1000')
        problem["output"] = generator(problem["prompt"],temperature=0.2,top_p=0.95,top_k=0,num_return_sequences=1,do_sample=True)[0]["generated_text"]
        new_data.append(problem)
        i += 1
    
    with open(f'save/{name}/_ds1000-all-completion.jsonl', 'w') as file:
        for entry in new_data:
            json_line = json.dumps(entry)
            file.write(json_line + "\n")


In [ ]:
# GENERATE OTHERS

for eval_type in eval_types:
    for name in names:
        if name.find("llama") >= 0:
            command = f'accelerate launch main.py --model={name_map[name]} --tasks={eval_type} --save_generations --save_generations_path=save/{name}/{reformatted_path} --n_samples=1 --allow_code_execution --trust_remote_code --prompt=continue --temperature=0.2 --generation_only'
            !{command}
        else:
            command = f'accelerate launch main.py --model={name_map["llama."+name]} --tasks={eval_type} --save_generations --save_generations_path=save/{name}/{reformatted_path} --peft_model={name_map[name]} --eos="</func>" --n_samples=1 --trust_remote_code --prompt=continue --temperature=0.2 --precision=fp16 --generation_only'
            !{command}

In [ ]:
# EVALUATE OTHERS

for eval_type in eval_types:
    
    for name in names:
        path = f"save/{name}/{reformatted_path}_{eval_type}.json"
        
        cmd_head = ""
        if eval_type.find("synthesize") < 0 or eval_type.find("ds1000") < 0:
            cmd_head = f"docker run -v $(pwd)/{path}:/app/{path}:ro -it  evaluation-harness-multiple"
            path = f"/app/{path}"
        else:
            cmd_head = "python3 main.py"
                
        if name.find("llama") >= 0:
            command = f'{cmd_head} --model={name_map[name]} --tasks={eval_type} --load_generations_path={path} --n_samples=1 --allow_code_execution --trust_remote_code --temperature=0.2 --prompt=continue'
            
            !{command}
            
            if eval_type=="humaneval": # or eval_type=="mbpp":
                command2 = f'{cmd_head} --model={name_map[name]} --tasks={eval_type+"plus"} --load_generations_path={path} --n_samples=1 --allow_code_execution --trust_remote_code --temperature=0.2'
                
                !{command2}
        else:               
                
            command = f'{cmd_head} --model={name_map["llama."+name]} --tasks={eval_type} --load_generations_path={path} --n_samples=1 --allow_code_execution --trust_remote_code --temperature=0.2 --peft_model={name_map[name]} --eos="</func>" --precision=fp16 --prompt=continue'
            
            !{command}
            
            if eval_type=="humaneval": # or eval_type=="mbpp":
                command2 = f'{cmd_head} --model={name_map[name]} --tasks={eval_type+"plus"} --load_generations_path={path} --n_samples=1 --allow_code_execution --trust_remote_code --temperature=0.2 --peft_model={name_map[name]} --eos="</func>" --precision=fp16 --prompt=continue'
                
                !{command2}
        